In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../../")

import torch
import torch.nn as nn
import torch.nn.functional as F
import math

DATA_PATH = "../data"

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

from src.log_mock import PrintLog
log = PrintLog()

import wandb

wandb.init(mode="disabled")

In [ ]:
wapi = wandb.Api()
runs = wapi.runs("bayes/iwildcam")

In [ ]:
for i, run in enumerate(runs):
    print(i, run.name)

In [ ]:
def create_reliability_plot(run, results_name):
    results = run.summary[results_name]
    print(f"sECE: {results['sece']:.4f}, ECE: {results['ece']:.4f}")
    bins = list(filter(lambda x: x[0] > 0, zip(results["bin_confidences"], results["bin_accuracies"], results["bin_counts"])))

    print("\\begin{tikzpicture}")
    print("    \\begin{axis}[calstyle, xmin=0, xmax=1, ymin=0, ymax=1]")
    print("        \\addplot[dashed, color=black] coordinates {(0,0) (1,1)};")
    print("        \\addplot[calline] coordinates {" + " ".join(map(lambda x: f"({x[0]}, {x[1]})", bins)) + "};")
    for conf, acc, count in list(bins):
        print(f"        \\node[above, anchor=south west, rotate=60, font=\\tiny] at (axis cs:{conf}, 1.0) {{{count}}};")
        print(f"        \\draw[dotted, color=black] (axis cs:{conf}, {acc}) -- (axis cs:{conf}, 1.0);")
    print("    \\end{axis}")
    print("\\end{tikzpicture}")


In [ ]:
import plotly.express as px
import pandas as pd
import dateutil
import datetime

def create_plot_data_for_run(run):
    parts = run.name.split("-")
    if len(parts) > 2:
        model_name = parts[0] + "-" + parts[1]
    else:
        model_name = parts[0]

    return {
        "model": model_name,
        "accuracy": run.summary["test_results"]["accuracy"],
        "macro f1": run.summary["test_results"]["macro_f1"],
        "log likelihood": run.summary["test_results"]["log_likelihood"],
        "ece": run.summary["test_results"]["ece"],
        "sece": run.summary["test_results"]["sece"],
        "id_val accuracy": run.summary["id_val_results"]["accuracy"],
        "id_val macro f1": run.summary["id_val_results"]["macro_f1"],
        "id_val log likelihood": run.summary["id_val_results"]["log_likelihood"],
        "id_val ece": run.summary["id_val_results"]["ece"],
        "id_val sece": run.summary["id_val_results"]["sece"],
    }

def plot(data, value):
    plot = px.box(data, x="model", y=value, color="model")
    return plot

def pareto_plot(data, x, y):
    plot = px.scatter(data, x=x, error_x=f"{x}_std", y=y, error_y=f"{y}_std", color="model")
    return plot

def build_data(runs):
    rows = []
    for run in runs:
        if dateutil.parser.parse(run.created_at) < datetime.datetime(2023, 3, 10, 10, 0):
            continue
        if run.state != "finished":
            continue
        if "old" in run.tags:
            print("Skipping old run " + run.name)
            continue
        if "test_results" not in run.summary:
            print("Skipping crashed run " + run.name)
            continue
        rows.append(create_plot_data_for_run(run))
    return pd.DataFrame.from_dict(rows)

def aggregate_data(data):
    aggregated_data = data.groupby(["model"]).agg({
        "model": "first",
        "accuracy": ["mean", "sem"], 
        "macro f1": ["mean", "sem"], 
        "log likelihood": ["mean", "sem"], 
        "sece": ["mean", "sem"],
        "ece": ["mean", "sem"],
        "id_val accuracy": ["mean", "sem"], 
        "id_val macro f1": ["mean", "sem"], 
        "id_val log likelihood": ["mean", "sem"], 
        "id_val sece": ["mean", "sem"],
        "id_val ece": ["mean", "sem"],
    })
    aggregated_data.columns = [a[0] + "_std" if a[1] == "sem" else a[0] for a in aggregated_data.columns.to_flat_index()]
    aggregated_data["accuracy_std"] *= 2
    aggregated_data["macro f1_std"] *= 2
    aggregated_data["log likelihood_std"] *= 2
    aggregated_data["sece_std"] *= 2
    aggregated_data["ece_std"] *= 2
    aggregated_data["id_val accuracy_std"] *= 2
    aggregated_data["id_val macro f1_std"] *= 2
    aggregated_data["id_val log likelihood_std"] *= 2
    aggregated_data["id_val sece_std"] *= 2
    aggregated_data["id_val ece_std"] *= 2
    return aggregated_data

In [ ]:
data = aggregate_data(build_data(runs))

In [ ]:
data

In [ ]:
algo_names = [
    ("map-1", "MAP"),
    ("map_5", "Deep Ensemble"),
    ("mcd-1", "MCD"),
    ("mcd_5", "MultiMCD"),
    ("swag-1", "SWAG"),
    ("swag_5", "MultiSWAG"),
    ("swag_ll-1", "LL SWAG"),
    ("laplace-1", "LL Laplace"),
    ("laplace_5", "LL MultiLaplace"),
    ("bbb-1", "LL BBB"),
    ("bbb_5", "LL MultiBBB"),
    ("rank1-1", "Rank-1 VI"),
    ("ll_ivon-1", "LL iVON"),
    ("ll_ivon_5", "LL MultiiVON"),
    ("svgd-1", "SVGD"),
    ("ll_svgd-1", "LL SVGD"),
    ("sngp", "SNGP"),
]

def num(value, std, best=None, ty=None):
    value = float(value)
    std = float(std)
    num_string = f"{value:.3f} \\pm {std:.3f}"

    if best is None or ty is None:
        return f"${num_string}$"

    if ty == "max":
        if value >= best:
            num_string = f"\\bm{{{num_string}}}"
    elif ty == "min":
        if value <= best:
            num_string = f"\\bm{{{num_string}}}"
    elif ty == "zero":
        if abs(value) <= best:
            num_string = f"\\bm{{{num_string}}}"
    return f"${num_string}$"

def col_name(name, align):
    return f"\\multicolumn{{1}}{{{align}}}{{{name}}}"

def create_table(data, prefix):
    print("\\begin{tabular}{l|rrrr}")
    print(f"    {col_name('Model', 'l')} & {col_name('Macro F1 Score', 'c')} & {col_name('Accuracy', 'c')} & {col_name('ECE', 'c')} & {col_name('sECE', 'c')} \\\\")
    print("    \\hline")

    best_f1, best_f1_std = 0, 0
    best_acc, best_acc_std = 0, 0
    best_ece, best_ece_std = 1000, 0
    best_sece, best_sece_std = 1000, 0

    for algo, name in algo_names:
        row = data[data["model"] == algo]
        
        if float(row[prefix + "macro f1"]) > best_f1:
            best_f1 = float(row[prefix + "macro f1"])
            best_f1_std = float(row[prefix + "macro f1_std"])
        
        if float(row[prefix + "accuracy"]) > best_acc:
            best_acc = float(row[prefix + "accuracy"])
            best_acc_std = float(row[prefix + "accuracy_std"])
        
        if float(row[prefix + "ece"]) < best_ece:
            best_ece = float(row[prefix + "ece"])
            best_ece_std = float(row[prefix + "ece_std"])
        
        if abs(float(row[prefix + "sece"])) < best_sece:
            best_sece = abs(float(row[prefix + "sece"]))
            best_sece_std = float(row[prefix + "sece_std"])

    best_f1 -= best_f1_std
    best_acc -= best_acc_std
    best_ece += best_ece_std
    best_sece = abs(best_sece) + best_sece_std

    for algo, name in algo_names:
        row = data[data["model"] == algo]
        print(f"    {name} & {num(row[prefix + 'macro f1'], row[prefix + 'macro f1_std'], best_f1, 'max')} & {num(row[prefix + 'accuracy'], row[prefix + 'accuracy_std'], best_acc, 'max')} & {num(row[prefix + 'ece'], row[prefix + 'ece_std'], best_ece, 'min')} & {num(row[prefix + 'sece'], row[prefix + 'sece_std'], best_sece, 'zero')} \\\\")
    print("\\end{tabular}")
create_table(data, "id_val ")

In [ ]:
data.to_csv(sep=",", header=True)

In [ ]:
pareto_plot(data, "macro f1", "sece")

In [ ]:
pareto_plot(data, "id_val macro f1", "id_val sece")